# Orlando - Regressão de 1 valor - Automóveis Cruzada

In [1]:
# ---------------------------------------
# 1. Bibliotecas
# ---------------------------------------
import pandas as pd
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras import backend as K

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from scikeras.wrappers import KerasRegressor

# ---------------------------------------
# 2. Tempo de execução
# ---------------------------------------
inicio = time.time()

# ---------------------------------------
# 3. Carregamento da base de dados
# ---------------------------------------
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')

# Remoção de colunas irrelevantes
colunas_remover = ['dateCrawled', 'dateCreated', 'nrOfPictures', 
                   'postalCode', 'lastSeen', 'name', 'seller', 'offerType']
base.drop(columns=colunas_remover, inplace=True)

# Remoção de outliers (preços absurdos)
base = base[(base['price'] > 100) & (base['price'] < 100000)]

# Preenchimento de valores nulos com valores comuns
valores = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzin',
    'notRepairedDamage': 'nein'
}
base.fillna(value=valores, inplace=True)

# ---------------------------------------
# 4. Separação entre X e y
# ---------------------------------------
X = base.iloc[:, 1:12].values  # exclui 'price'
y = base.iloc[:, 0].values     # apenas 'price'

# ---------------------------------------
# 5. OneHotEncoder com saída densa
# ---------------------------------------
categoricas = [0, 1, 3, 5, 8, 9, 10]
transformador = ColumnTransformer(transformers=[
    ("OneHot", OneHotEncoder(sparse_output=False), categoricas)
], remainder='passthrough')

X = transformador.fit_transform(X)

# ---------------------------------------
# 6. Normalização dos dados numéricos
# ---------------------------------------
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# ---------------------------------------
# 7. Definição da rede
# ---------------------------------------
def criar_rede():
    K.clear_session()
    modelo = Sequential([
        InputLayer(input_shape=(X.shape[1],)),
        Dense(units=158, activation='relu'),
        Dense(units=158, activation='relu'),
        Dense(units=1, activation='linear')
    ])
    modelo.compile(optimizer='adam',
                   loss='mean_absolute_error',
                   metrics=['mean_absolute_error'])
    return modelo

# ---------------------------------------
# 8. Empacotamento com Scikeras
# ---------------------------------------
regressor = KerasRegressor(
    model=criar_rede,
    epochs=100,
    batch_size=300,
    verbose=0
)

# ---------------------------------------
# 9. Validação cruzada com 5 folds
# ---------------------------------------
resultados = cross_val_score(
    estimator=regressor,
    X=X,
    y=y,
    cv=5,
    scoring='neg_mean_absolute_error'
)

# ---------------------------------------
# 10. Exibição dos resultados
# ---------------------------------------
maes = -resultados  # transformando para valores positivos

print(f"\n MAEs por fold: {np.round(maes, 2)}")
print(f" MAE médio: {maes.mean():.2f}")
print(f" Desvio padrão: {maes.std():.2f}")

fim = time.time()
print(f"\n⏱️ Tempo total de execução: {(fim - inicio)/60:.2f} minutos")


c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



 MAEs por fold: [1987.48 2000.76 1953.   1990.5  1948.47]
 MAE médio: 1976.04
 Desvio padrão: 21.17

⏱️ Tempo total de execução: 18.96 minutos
